In [1]:
import tensorflow as tf
tf.enable_eager_execution
from tensorflow import keras
from tensorflow.keras import layers,Sequential
from tensorflow.keras.utils import to_categorical
import numpy as np

In [12]:
class Shared_actor_critic :
    def __init__(self,state_size=11,action_size=20,seq_len=15,h1=32,h2=64):
        
        self.state_size =state_size
        self.action_size = action_size
        self.seq_len = seq_len
        self.state_shape=(1,seq_len,state_size)
        self.reward_shape = (1,seq_len,1)
        self.action_shape = (1,seq_len,action_size)
        
        ###creation of layers and model
        ##batch_input_shape=(numberofSequence,timesteps,data_dim)
        self.l1 = layers.LSTM(units=h1,return_sequences=True,batch_input_shape=(None,3,5))
        self.l2 = layers.LSTM(units=h2,return_sequences=True)
        self.l3 = layers.Dense(2,activation='softmax')
        self.l4 = layers.Dense(1)
        self.Actor = Sequential([self.l1,self.l2,self.l3])
        self.critic = Sequential([self.l1,self.l2,self.l4])
        self.critic.compile(loss='mse',optimizer='sgd')
        self.advantage = np.zeros((seq_len,))
        self.loss_fn = self.actor_loss(self.advantage)
        self.Actor.compile(loss =self.loss_fn,optimizer='sgd' )
    
    def mult_loss_by_advantage(self, y_true,y_pred,advantage):
        ###change size of advantage compatible to y_pred
        #then multiply
        self.advantage = self.advantage.reshape((self.seq_len,1))
        temp = np.ones((1,self.action_size))
        advantage = np.matmul(self.advantage,temp)
        print("multiplication done")
        self.advantage = self.advantage.reshape((1,self.seq_len,self.action_size))
        self.loss1 = (tf.log(y_pred)) * (advantage)
        print(loss1)
        return np.sum(self.loss1)

    def actor_loss(self,advantage):
        #y_act is a one hot vector, actual action that was taken
        #y_pred is softmax output from actor model
        def my_fun(y_true,y_pred):
            return self.mult_loss_by_advantage(y_true,y_pred,advantage)
        return my_fun
    ### helper functions
    def cal_discounted_rew(self, rew, disc_factor = 0.9):
        for i in range(rew.shape[0]-2,-1,-1):
            rew[i] += disc_factor *rew[i+1]
        return rew

    def cal_advantage(self, reward_array, state_array,disc_factor=0.9):
        ###parametrize it for generalized advantage estimation
        values_of_states = self.critic.predict(state_array)
        print("ok1")
        ###reshape this array values_of_states
        values_of_states = values_of_states.reshape((values_of_states.shape[1],))
        reward_array = cal_discounted_rew(reward_array,disc_factor)    
        print("ok2")
        advantage_array = np.zeros(reward_array.shape)
        advantage_array = reward_array - values_of_states
        print("ok3")
        return advantage_array
    
    def fit(self,state_array, action_array, rew_array):
        state_array = state_array.reshape(self.state_shape)
        action_array = to_categorical(action_array, num_classes = self.action_size)
        action_array = action_array.reshape(self.action_shape)
        
        rew_array_for_critic = self.cal_discounted_rew(rew_array)
        rew_array_for_critic = rew_array_for_critic.reshape(self.reward_shape)
        self.critic.fit(state_array,rew_array_for_critic)
        adv = self.cal_advantage(rew_array,state_array)
        
        self.Actor.fit(state_array,action_array)   

In [13]:
A_C_net = Shared_actor_critic(5,2,3)
in_data =np.random.normal(1,2,(3,5))
y = np.random.normal(1,0,(3,))
act_arr = np.asarray([1,2,1])

multiplication done


ValueError: cannot reshape array of size 3 into shape (1,3,2)